<a href="https://colab.research.google.com/github/thekakol/AI_MED_Recruitment/blob/main/AI_Med.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview of the asignment
TF I'm trying to achieve here?

# Required libraries

In [4]:
import numpy
import pandas
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
#Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Creating the Data table

In [5]:
#Three columns are saved as string datatype, however they contain float data that can be useful for the model
#This motivates us to change these columns form string into float
task_data = pandas.read_csv('task_data.csv', delimiter=',')
print(task_data.dtypes, "\n")
for string in ('CTR - Cardiothoracic Ratio','Inscribed circle radius','Heart perimeter'):
  task_data[string] = task_data[string].str.replace(',','.')
  task_data[string] = task_data[string].astype(float)
print(task_data.dtypes, "\n")
task_data

ID                              int64
Cardiomegaly                    int64
Heart width                     int64
Lung width                      int64
CTR - Cardiothoracic Ratio     object
xx                            float64
yy                            float64
xy                            float64
normalized_diff               float64
Inscribed circle radius        object
Polygon Area Ratio            float64
Heart perimeter                object
Heart area                      int64
Lung area                       int64
dtype: object 

ID                              int64
Cardiomegaly                    int64
Heart width                     int64
Lung width                      int64
CTR - Cardiothoracic Ratio    float64
xx                            float64
yy                            float64
xy                            float64
normalized_diff               float64
Inscribed circle radius       float64
Polygon Area Ratio            float64
Heart perimeter               floa

,ID,Cardiomegaly,Heart width,Lung width,CTR - Cardiothoracic Ratio,xx,yy,xy,normalized_diff,Inscribed circle radius,Polygon Area Ratio,Heart perimeter,Heart area,Lung area
0,1,0,172,405,0.424691,1682.360871,3153.671880,-638.531109,-0.304239,68.81860,0.213446,679.487369,24898,75419
1,2,1,159,391,0.406650,1526.660960,5102.159054,-889.678405,-0.539387,73.92564,0.203652,788.658942,29851,94494
2,5,0,208,400,0.520000,2465.903392,5376.834707,-1755.344699,-0.371163,69.33974,0.320787,862.322937,33653,66666
3,7,1,226,435,0.519540,2509.063593,6129.821270,-1025.079806,-0.419123,84.14868,0.317545,906.724959,42018,82596
4,8,1,211,420,0.502381,2368.770135,5441.767075,-1493.040062,-0.393442,73.78347,0.263542,864.239678,35346,85631
5,9,1,222,405,0.548148,2351.057355,8378.677729,-812.061371,-0.561768,83.86298,0.328101,1001.068103,46381,92755
6,10,1,202,498,0.405622,2251.879460,4467.406612,-511.955541,-0.329727,84.72308,0.209396,808.298550,35417,116542
7,11,1,228,474,0.481013,2971.936804,3971.943088,-885.822012,-0.144013,83.73769,0.242418,821.955404,36716,97325
8,12,0,176,449,0.391982,1833.841218,3709.724540,-831.472858,-0.338389,73.40981,0.181388,727.972650,27939,99656
9,13,1,223,473,0.471459,2412.707474,6169.769803,-880.450924,-0.437760,76.41989,0.261665,922.925968,38162,105442


# Preparing data (to make something useful out of this)

In [6]:
usefulData = task_data[["Heart width", "Lung width", "xx","yy","xy","normalized_diff","Inscribed circle radius", "Polygon Area Ratio", "Heart perimeter", "Heart area ","Lung area" ]]
usefulData

,Heart width,Lung width,xx,yy,xy,normalized_diff,Inscribed circle radius,Polygon Area Ratio,Heart perimeter,Heart area,Lung area
0,172,405,1682.360871,3153.671880,-638.531109,-0.304239,68.81860,0.213446,679.487369,24898,75419
1,159,391,1526.660960,5102.159054,-889.678405,-0.539387,73.92564,0.203652,788.658942,29851,94494
2,208,400,2465.903392,5376.834707,-1755.344699,-0.371163,69.33974,0.320787,862.322937,33653,66666
3,226,435,2509.063593,6129.821270,-1025.079806,-0.419123,84.14868,0.317545,906.724959,42018,82596
4,211,420,2368.770135,5441.767075,-1493.040062,-0.393442,73.78347,0.263542,864.239678,35346,85631
5,222,405,2351.057355,8378.677729,-812.061371,-0.561768,83.86298,0.328101,1001.068103,46381,92755
6,202,498,2251.879460,4467.406612,-511.955541,-0.329727,84.72308,0.209396,808.298550,35417,116542
7,228,474,2971.936804,3971.943088,-885.822012,-0.144013,83.73769,0.242418,821.955404,36716,97325
8,176,449,1833.841218,3709.724540,-831.472858,-0.338389,73.40981,0.181388,727.972650,27939,99656
9,223,473,2412.707474,6169.769803,-880.450924,-0.437760,76.41989,0.261665,922.925968,38162,105442


In [7]:
result = task_data['Cardiomegaly']
result

,Cardiomegaly
0,0
1,1
2,0
3,1
4,1
5,1
6,1
7,1
8,0
9,1


# Dividing data into train and test

In [8]:
trainData, testData, trainResult, testResult = train_test_split(usefulData, result, test_size=0.2, random_state=69)

# Weird shit starts here
No one will stop me from optimising everything

# WE CAN GO BIGGER
We'll apply all possible methods, because no one will stop me

# **Universal Optimatisation Functions**

In [9]:
def pipeOptimiser(classificator, parameters):
  rskf = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=20,
    random_state=None
  )

  pipe = Pipeline(steps=[
      ("scaler", StandardScaler()),
      ("model", classificator())
  ])
  test = classificator()
  grid_search = GridSearchCV(
    estimator=pipe,

    param_grid=parameters,
    scoring="accuracy",
    cv=rskf,
    verbose=1,
    n_jobs=-1
  )
  grid_search.fit(trainData, trainResult)
  return ( grid_search.best_params_,grid_search.best_score_)

#K - nearest neighbors



In [10]:
#Finding best parameters for K - nearest neighbors
param_grid = {
    "model__n_neighbors": [1,2,3,4],
    "model__weights": ["uniform", "distance"],
    "model__metric": ["minkowski", "manhattan", "euclidean", "chebyshev"],
}
best_params_pipe_knn, best_score = pipeOptimiser(KNeighborsClassifier, param_grid)
print(f"Best parameters: {best_params_pipe_knn}")
print(f"Best accuracy (averaged CV): {best_score:.4f}")

Fitting 100 folds for each of 32 candidates, totalling 3200 fits
Best parameters: {'model__metric': 'manhattan', 'model__n_neighbors': 4, 'model__weights': 'distance'}
Best accuracy (averaged CV): 0.8530


In [11]:
#Testing the train data set with found parameters
pipe_knn = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", KNeighborsClassifier(
        n_neighbors = best_params_pipe_knn['model__n_neighbors'],
        weights     = best_params_pipe_knn['model__weights'],
        metric      = best_params_pipe_knn['model__metric'],
    ))
])
pipe_knn.fit(trainData, trainResult)
cv_score = numpy.round(cross_val_score(pipe_knn, trainData, trainResult), 2)
print("Scores of training data cross-validation (each fold):")
list(map(print, cv_score))
print(f"\nCross-validation mean score: {numpy.mean(cv_score):.3}")
print(f"Standard deviation of CV score: {numpy.std(cv_score):.3f}")

Scores of training data cross-validation (each fold):
0.83
1.0
0.67
0.83
1.0

Cross-validation mean score: 0.866
Standard deviation of CV score: 0.124


# Support Vector Machine

In [12]:
#Finding best parameters for Support Vector Machine
param_grid = {
    "model__kernel":["linear", "poly", "rbf", "sigmoid"],
    "model__C":[1,2,3,4],
    "model__gamma":["scale","auto"],
    "model__class_weight":["balanced",None]
}
best_params_pipe_svc, best_score = pipeOptimiser(SVC, param_grid)
print(f"Best parameters: {best_params_pipe_svc}")
print(f"Best accuracy (averaged CV): {best_score:.4f}")

Fitting 100 folds for each of 64 candidates, totalling 6400 fits
Best parameters: {'model__C': 1, 'model__class_weight': 'balanced', 'model__gamma': 'scale', 'model__kernel': 'rbf'}
Best accuracy (averaged CV): 0.8737


In [13]:
pipe_svc = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", SVC(                  # Support Vector Classifier
        kernel=best_params_pipe_svc["model__kernel"],               # RBF kernel captures non-linear decision boundaries
        C=best_params_pipe_svc["model__C"],                        # Regularization strength (higher = tighter fit to training data)
        gamma=best_params_pipe_svc["model__gamma"],              # Kernel width; 'scale' adapts to data variance
        class_weight=best_params_pipe_svc["model__class_weight"]           # Treat classes equally (no re-weighting)
    ))
])

pipe_svc.fit(trainData, trainResult)

cv_score = numpy.round(cross_val_score(pipe_svc, trainData, trainResult), 2)

print("Scores of training data cross-validation (each fold):")
list(map(print, cv_score))
print(f"\nCross-validation mean score: {cv_score.mean():.3f}")
print(f"Standard deviation of CV score: {cv_score.std():.3f}")

Scores of training data cross-validation (each fold):
1.0
1.0
0.67
0.83
1.0

Cross-validation mean score: 0.900
Standard deviation of CV score: 0.133


# Logistic Regression

In [14]:
#Finding best parameters for Logistic Regression
param_grid = {
  "model__C":[1,2,3],
  "model__penalty":["l1", "l2", "elasticnet", None],
  "model__solver":["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
  "model__max_iter":[1000,10000],
  "model__class_weight":["balanced",None],
}
best_params_pipe_log, best_score = pipeOptimiser(LogisticRegression, param_grid)
print(f"Best parameters: {best_params_pipe_log}")
print(f"Best accuracy (averaged CV): {best_score:.4f}")

Fitting 100 folds for each of 288 candidates, totalling 28800 fits
Best parameters: {'model__C': 1, 'model__class_weight': None, 'model__max_iter': 1000, 'model__penalty': 'l1', 'model__solver': 'saga'}
Best accuracy (averaged CV): 0.7880


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
13200 fits failed out of a total of 28800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt,

In [15]:
pipe_log = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(
        C=best_params_pipe_log["model__C"],
        penalty=best_params_pipe_log["model__penalty"],
        solver=best_params_pipe_log["model__solver"],
        max_iter=best_params_pipe_log["model__max_iter"],
        class_weight=best_params_pipe_log["model__class_weight"]
    ))
])

pipe_log.fit(trainData, trainResult)

cv_score = numpy.round(cross_val_score(pipe_log, trainData, trainResult), 2)

print("Scores of training data cross-validation (each fold):")
list(map(print, cv_score))
print(f"\nCross-validation mean score: {cv_score.mean():.3f}")
print(f"Standard deviation of CV score: {cv_score.std():.3f}")

Scores of training data cross-validation (each fold):
0.83
0.83
0.5
0.83
0.8

Cross-validation mean score: 0.758
Standard deviation of CV score: 0.130


# Running the Model on the Test Dataset

In [16]:
#Predicting for all models
y_pred_knn  = pipe_knn.predict(testData)
y_pred_svc  = pipe_svc.predict(testData)
y_pred_log  = pipe_log.predict(testData)

#Evaluating all models
#Accuracy
acc_knn  = accuracy_score(testResult, y_pred_knn)
acc_svc  = accuracy_score(testResult, y_pred_svc)
acc_log  = accuracy_score(testResult, y_pred_log)
#Precision
prec_knn  = precision_score(testResult, y_pred_knn)
prec_svc  = precision_score(testResult, y_pred_svc)
prec_log  = precision_score(testResult, y_pred_log)
#Recall
recall_knn  = recall_score(testResult, y_pred_knn)
recall_svc  = recall_score(testResult, y_pred_svc)
recall_log  = recall_score(testResult, y_pred_log)
#F1
f1_knn  = f1_score(testResult, y_pred_knn)
f1_svc  = f1_score(testResult, y_pred_svc)
f1_log  = f1_score(testResult, y_pred_log)
print(f"Accuracy on test set:")
print(f"- Accuracy of KNN Classifier model on test dataset:                     {acc_knn:.4f}")
print(f"- Accuracy of SVC model on test dataset:                                {acc_svc:.4f}")
print(f"- Accuracy of Logistic Regression model on test dataset:                {acc_log:.4f}")
print(f"\n Precision on test set:")
print(f"- Precision of KNN Classifier model on test dataset:                    {prec_knn:.4f}")
print(f"- Precision of SVC model on test dataset:                               {prec_svc:.4f}")
print(f"- Precision of Logistic Regression model on test dataset:               {prec_log:.4f}")
print(f"\n Recall on test set:")
print(f"- Recall of KNN Classifier model on test dataset:                       {recall_knn:.4f}")
print(f"- Recall of SVC model on test dataset:                                  {recall_svc:.4f}")
print(f"- Recall of Logistic Regression model on test dataset:                  {recall_log:.4f}")
print(f"\n F1 on test set:")
print(f"- F1 of KNN Classifier model on test dataset:                           {f1_knn:.4f}")
print(f"- F1 of SVC model on test dataset:                                      {f1_svc:.4f}")
print(f"- F1 of Logistic Regression model on test dataset:                      {f1_log:.4f}")
print(f"Confusion matrix on test data")
print(f"- Confusion matrix of KNN Classifier model on test dataset:             {confusion_matrix(testResult,y_pred_knn)}")
print(f"- Confusion matrix of SVC model on test dataset:                        {confusion_matrix(testResult,y_pred_svc)}")
print(f"- Confusion matrix of Logistic Regression model on test dataset:        {confusion_matrix(testResult,y_pred_log)}")

Accuracy on test set:
- Accuracy of KNN Classifier model on test dataset:                     0.7500
- Accuracy of SVC model on test dataset:                                0.6250
- Accuracy of Logistic Regression model on test dataset:                0.6250

 Precision on test set:
- Precision of KNN Classifier model on test dataset:                    0.8333
- Precision of SVC model on test dataset:                               0.8000
- Precision of Logistic Regression model on test dataset:               0.8000

 Recall on test set:
- Recall of KNN Classifier model on test dataset:                       0.8333
- Recall of SVC model on test dataset:                                  0.6667
- Recall of Logistic Regression model on test dataset:                  0.6667

 F1 on test set:
- F1 of KNN Classifier model on test dataset:                           0.8333
- F1 of SVC model on test dataset:                                      0.7273
- F1 of Logistic Regression model on test da